# Personalizované dispersion ploty s Altair a Pandas

*Dispersion plot* umožňuje vizualizovat disperzi = rozložení vybraných jevů v textu či korpusu textů. V NLTK je dostupný ve funkci `nltk.draw.dispersion_plot()` a jako metoda na objektech typu `Text`.

In [ ]:
from nltk.book import *

Můžeme se např. podívat na disperzi určitých klíčových slov v projevech amerických prezidentů od George Washingtona (1789) po současnost. Ty jsou uložené v proměnné `text4` (byť je to přísně vzato vlastně korpus textů, ale nic nám nebrání s ním pracovat jako s textem jedním):

In [ ]:
text4.dispersion_plot([
    "citizen",
    "democracy",
    "freedom",
    "duty",
    "America"
])

Pro základní rychlou vizualizaci poslouží velmi dobře, ale při interpretaci můžou vyvstat dodatečné otázky, na které není z tohoto grafu úplně snadné odpovědět -- kterým historickým obdobím odpovídají hustší či naopak řidší výskyty jednotlivých slov? Kterým konkrétním prezidentským projevům?

V tom výše zobrazeném grafu můžeme přinejlepším hádat. Ale nic nám nebrání si v této chvíli vyrobit vlastní vylepšenou verzi!

(To neznamená, že ta existující verze z NLTK je špatná -- slouží pro rychlou orientaci, někdy může plně stačit, jen v tomto případě jsme zjistili, že některé potenciálně zajímavé údaje z grafu nevyčteme, takže dává smysl věnovat trochu víc času tomu, abychom si udělali vlastní na míru.)

In [ ]:
import pandas as pd
import altair as alt
from nltk.corpus import inaugural

Korpus `inaugural` má stejný obsah jako `text4`...

In [ ]:
text4[:10]

In [ ]:
len(text4)

In [ ]:
inaugural.words()[:10]

In [ ]:
len(inaugural.words())

... ale má metadata navíc. Projevy v něm nejsou jen pospojované dohromady, můžeme si je vyvolat i jednotlivě a máme informace o jejich autorech a rocích:

In [ ]:
inaugural.fileids()[:5]

In [ ]:
prefixes = ("citizen", "democra", "free", "duti", "duty", "america")

In [ ]:
words = []
categories = []
times = []
presidents = []
years = []
even = []
for i, fileid in enumerate(inaugural.fileids()):
    year = int(fileid[:4])
    pres = fileid.split("-")[1].split(".")[0]
    fileid_words = inaugural.words(fileid)
    total = len(fileid_words)
    for index, word in enumerate(fileid_words):
        cf = word.casefold()
        if cf.startswith(prefixes) and not cf.startswith("freezing"):
            words.append(word)
            cat = cf[:4]
            cat = "duty" if cat == "duti" else cat
            categories.append(cat)
            times.append(year + index/total*4)
            presidents.append(pres)
            years.append(year)
            even.append(i % 2 == 0)

In [ ]:
df = pd.DataFrame(dict(
    time=times,
    category=categories,
    word=words,
    president=presidents,
    year=years,
    even=even
))
df.head()

In [ ]:
df.tail()

Základní specifikace grafu, která říká jen to, co a kde chceme zobrazit, bez dalšího ladění:

In [ ]:
alt.Chart(df).mark_tick().encode(
    x="time",
    y="category",
    color="even",
    tooltip=["president", "year", "word"]
).interactive()

Vyladěná specifikace grafu, kde jsme navíc [upravili i způsob zobrazení některých prvků](https://altair-viz.github.io/user_guide/configuration.html), aby lépe vypadaly, a změnili popisky:

In [ ]:
alt.Chart(df).mark_tick().encode(
    alt.X("time", scale=alt.Scale(domain=(1789, 2013)), title="Rok"),
    alt.Y("category", title="Kategorie slova"),
    alt.Color("even", legend=None),
    tooltip=["president", "year", "word"]
).properties(
    title="Disperze vybraných kategorií slov v inauguračních projevech amerických prezidentů",
    width=800,
    height=400
).configure(
    numberFormat="d"
).configure_title(
    fontSize=18
).configure_axis(
    labelFontSize=16,
    titleFontSize=18
).configure_tick(
    size=40
).interactive()

Pokud vás knihovna pro vizualizaci dat Altair zaujala, víc o ní se dozvíte v její dokumentaci: <https://altair-viz.github.io/>. Přidám ještě osobní tip, jak s dokumentací a potažmo celou knihovnou pracovat -- obsahuje totiž velké množstí informací a funkcionality. Nemá smysl se nejdřív snažit všechno přečíst a naučit, nelze to udržet v hlavě. Sám knihovnu taky nejenže neumím zpaměti, ale když ji delší dobu nepoužiju, tak zapomenu i základy. Výše uvedený kód pro *dispersion plot* jsem nevysypal jen tak z rukávu, to by možná zvládl někdo, kdo Altair používá dennodenně. Mně to zabralo notnou dávku googlení a pročítání dokumentace, ale díky tomu, že jsem měl v hlavě konkrétní cíl, jsem se v tom moři informací neutopil.

Takže doporučuju následující postup: projděte si [úvodní tutorial](https://altair-viz.github.io/getting_started/starting.html), který vám pomůže se základní orientací, a pak se pusťte do tvorby vlastních grafů, podle vašich zájmů a potřeb. Při tom vycházejte z [příkladů v galerii](https://altair-viz.github.io/gallery/index.html). V galerii velmi pravděpodobně najdete grafy podobné těm, které budete chtít vytvářet, a k nim jejich zdrojový kód. Zkopírujte si ho, zkuste pozměnit nějaké drobnosti, abyste si ho trochu osahali, a nakonec ho upravte pro svoje vlastní data. Do zbývajících témat v dokumentaci nahlížejte hlavně podle potřeby, když narazíte na nějakou překážku, kvůli níž je nutné si nějakou oblast nastudovat blíže. Ideálně se k nim nechte navést Googlem, protože ten vám poradí i případné alternativní zdroje, které mohou být pro vaši otázku relevantnější než oficiální dokumentace (např. StackOverflow).

A když už jsme u toho -- tenhle přístup není relevantní jen pro Altair. Analogický postup platí pro jakoukoli větší knihovnu, s níž se chcete naučit pracovat, a nakonec i pro celé programovací jazyky. Nečekejte, až budete Python "umět", pusťte se do různých drobných projektů rovnou, protože s jejich pomocí se ho naučíte mnohem rychleji a spolehlivěji :)

V tuto chvíli bohužel v Altairu nelze změnit velikost všech prvků v grafu nějak plošně, je potřeba prostřednictvím konfigurace (viz volání těch různých metod `.configure_...()` výše) změnit velikost každého prvku, kde je to třeba, individuálně. Doufejme, že plošné škálování celého grafu bude výhledově implementováno, viz <https://github.com/vega/vega/issues/2946>.

Pokud chcete v jednom notebooku dělat grafů víc, vadí vám malá defaultní velikost a chcete se vyhnout tomu, abyste museli u každého grafu konfiguraci specifikovat znovu a znovu, můžete si nastavit vlastní *theme* s většími velikostmi inkriminovaných prvků oproti defaultu:

In [ ]:
def my_theme():
    return {
        "config": {
            "numberFormat": "d",
            "axis": {
                "labelFontSize": 16,
                "titleFontSize": 18,
            },
            "header": {
                "labelFontSize": 16,
                "titleFontSize": 18,
            },
            "title": {
                "fontSize": 18,
            },
            "tick": {
                "size": 40,
            },
        },
    }
alt.themes.register("my_theme", my_theme)
alt.themes.enable("my_theme")

In [ ]:
alt.Chart(df).mark_tick().encode(
    alt.X("time", scale=alt.Scale(domain=(1789, 2013)), title="Rok"),
    alt.Y("category", title="Kategorie slova"),
    alt.Color("even", legend=None),
    tooltip=["president", "year", "word"]
).properties(
    title="Disperze vybraných kategorií slov v inauguračních projevech amerických prezidentů",
    width=800,
    height=400
).interactive()

Výsledek je stejný jako u předchozího grafu, navzdory tomu, že jsme tentokrát na objektu typu `Chart` neprovedli žádnou explicitní konfiguraci; místo toho byla načtena z globální konfigurace stanovené přes `my_theme`.

Pokud se chystáte obrázek použít v publikaci, tak alternativou může být ho nechat v defaultní podobě, exportovat ve formátu SVG a naškálovat v nějakém samostatném grafickém softwaru.

In [ ]:
alt.themes.enable("default")

chart = alt.Chart(df).mark_tick().encode(
    alt.X("time", scale=alt.Scale(domain=(1789, 2013)), title="Rok"),
    alt.Y("category", title="Kategorie slova"),
    alt.Color("even", legend=None),
    tooltip=["president", "year", "word"]
).properties(
    title="Disperze vybraných kategorií slov v inauguračních projevech amerických prezidentů",
    width=400,
    height=200
).configure(numberFormat="d").interactive()
chart

In [ ]:
chart.save("disperze.svg")

V [dokumentaci Altairu](https://altair-viz.github.io/user_guide/saving_charts.html#figure-size-resolution) taky narazíte na následující možnosti škálování celého grafu při exportu:

```python
alt.renderers.set_embed_options(scaleFactor=2)
```

Nebo:

```python
chart.save("06-disperze.png", scale_factor=2)
```

Ty bohužel na Jupyteru [z technických důvodů](https://github.com/altair-viz/altair_saver/issues/39) nefungují.